In [1]:
import os, mne, socket
from mne_bids import BIDSPath, read_raw_bids
import pandas as pd
import nibabel as nib

%matplotlib qt

# Get the hostname
hostname = socket.gethostname()

if 'hpc' in hostname:
    # You are running your code on HPC
    curr_dir = os.getcwd()
    path_parts = curr_dir.split(os.sep)
    dataPath = os.path.join(os.sep, 'scratch', 'work', 'courses', 'PSYCH-GA-3405-2024fa')
elif hostname == 'sebastian_mac':
    # You are Sebastian
    # Setting up paths for data
    mydir = os.getcwd()
    path_parts = mydir.split(os.sep)
    idcs = [i for i, c in enumerate(mydir) if c == os.sep]
    dataPath = mydir[:idcs[-2]]
else:
    # You are running on your device with Google drive path
    # Define the paths and initialize Fieldtrip
    my_user_id = 'mdd9787'  # change this to your netID
    curr_dir = os.getcwd()
    path_parts = curr_dir.split(os.sep)
    base_dir = os.path.join(os.sep, *path_parts[:3])
    dataPath = os.path.join(base_dir, 'Library', 'CloudStorage', f'GoogleDrive-{my_user_id}@nyu.edu', 'My Drive', 'Coursework', 'EEG MEG methods', 'ClassData')

eegRoot = os.path.join(dataPath, 'EEGBids')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Installing Freesurfer
##### In order to perform anatomical processing for headmodeling and source reconstruction, we will be using Freesurfer run_recon_all.
##### First step is to install freesurfer on your devies. You can find the installation best suited for your device:
##### https://surfer.nmr.mgh.harvard.edu/fswiki/rel7downloads
##### And follow the instructions to set freesurfer: https://surfer.nmr.mgh.harvard.edu/fswiki/DownloadAndInstall?action=AttachFile&do=get&target=installFS_demo.mp4


In [2]:
## Load data
groupName = 'GroupD'  # Change group name to analyze different dataset (valid IDs: GroupA, GroupC, GroupD)
userName = 'mrugank'  # Make sure to ensure that you are writing to your derivatives 
taskName = 'oddball'

# Get subject code for your group
groupMap = {'GroupA': '001', 'GroupC': '003', 'GroupD': '004'}
subjCode = groupMap[groupName]

dataPath = os.path.join(eegRoot, f'sub-{subjCode}', 'eeg')
derivPath = os.path.join(eegRoot, 'derivatives', userName, 'preprocessing', f'sub-{subjCode}')

if not os.path.exists(derivPath):
    os.makedirs(derivPath)

saveRoot = f'sub-{subjCode}_task-{taskName}_'

# Load the data
raw = read_raw_bids(BIDSPath(subject=subjCode, task=taskName, root=eegRoot))

Extracting EDF parameters from /Users/mrugankdake/Library/CloudStorage/GoogleDrive-mdd9787@nyu.edu/My Drive/Coursework/EEG MEG methods/ClassData/EEGBids/sub-004/eeg/sub-004_task-oddball_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /Users/mrugankdake/Library/CloudStorage/GoogleDrive-mdd9787@nyu.edu/My Drive/Coursework/EEG MEG methods/ClassData/EEGBids/sub-004/eeg/sub-004_task-oddball_events.tsv.
The event "" refers to multiple event values. Creating hierarchical event names.
    Renaming event:  -> /4
    Renaming event:  -> /2
    Renaming event:  -> /4
    Renaming event:  -> /1
    Renaming event:  -> /4
    Renaming event:  -> /2
    Renaming event:  -> /1
    Renaming event:  -> /1
    Renaming event:  -> /1
    Renaming event:  -> /4
    Renaming event:  -> /2
    Renaming event:  -> /2
    Renaming event:  -> /8
    Renaming event:  -> /1
    Renaming event:  -> /4
    Renaming event:  -> /2
    Renaming event: 

/var/folders/0w/p5w49zc93976_p2hxj1zws2w0000gn/T/ipykernel_60909/3064140265.py:19: RuntimeWarning: The number of channels in the channels.tsv sidecar file (261) does not match the number of channels in the raw data file (272). Will not try to set channel names.
  raw = read_raw_bids(BIDSPath(subject=subjCode, task=taskName, root=eegRoot))
/var/folders/0w/p5w49zc93976_p2hxj1zws2w0000gn/T/ipykernel_60909/3064140265.py:19: RuntimeWarning: The unit for channel(s) Status has changed from NA to V.
  raw = read_raw_bids(BIDSPath(subject=subjCode, task=taskName, root=eegRoot))
/var/folders/0w/p5w49zc93976_p2hxj1zws2w0000gn/T/ipykernel_60909/3064140265.py:19: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['EXG1', 'EXG2', 'EXG3', 'EXG4']
  raw = read_raw_bids(BIDSPath(subject=subjCode, task=taskName, root=eegRoot))
/var/folders/0w/p5w49zc93976_p2hxj1zws2w0000gn/T/ipykernel_60909/3064140265.py:19: RuntimeWarning: DigMontage is only a subset of info. There 

# Create headmodel from anatomical

In [ ]:
# Read MRI data
mriPath = os.path.join(eegRoot, f'sub-{subjCode}', 'anat', f'sub-{subjCode}_T1w.nii.gz')
mri_img = nib.load(mriPath)

# Convert the nibabel image into MNE format (mne does not directly read .nii.gz)
# You need to handle the affine and data of the MRI image separately
mri_data = mri_img.get_fdata()  # this is the MRI data (3D array)
affine = mri_img.affine  # transformation matrix for MRI space

# Create an MNE Info object if you need to work with data objects like Raw or Epochs
# info = mne.create_info(ch_names=[''] * 0, ch_types=['eeg'] * 0)  # Placeholder
mri = mne.preprocessing.read_mri(mriPath)

AttributeError: No mne.preprocessing attribute read_mri

: 

In [ ]:
anatPath = os.path.join(eegRoot, f'sub-{subjCode}', 'anat')
# Read headshape file
tbl = pd.read_table(os.path.join(anatPath, f'sub-{subjCode}_headshape.tsv'), keep_default_na=False, names=['Label', 'X', 'Y', 'Z'], skiprows=1)

extra_points = tbl[tbl['Label'] == 'EXTRA'][['X', 'Y', 'Z']].values

# Access the current montage in raw
montage = raw.get_montage()

# Create new entries for the headshape points
# The headshape points will be stored as extras in the montage's 'dig' field
new_dig = [
    dict(
        kind=mne.io.constants.FIFF.FIFFV_POINT_EXTRA,  # Headshape point kind
        ident=i + len(montage.dig),                 # Unique identifier
        r=pos,                                      # (x, y, z) coordinates
        coord_frame=mne.io.constants.FIFF.FIFFV_COORD_HEAD
    )
    for i, pos in enumerate(extra_points)
]

# Add the new headshape points to the existing montage
montage.dig.extend(new_dig)

# Set the updated montage back to the raw object
raw.set_montage(montage)



<RawEDF | sub-004_task-oddball_eeg.bdf, 272 x 2347008 (4584.0 s), ~467 kB, data not loaded>

In [12]:
# Visualize the montage
raw.plot_sensors(show_names=False)
fig = mne.viz.plot_montage(raw.info, show_names=False, kind='3d')

/var/folders/0w/p5w49zc93976_p2hxj1zws2w0000gn/T/ipykernel_60715/1267379592.py:2: RuntimeWarning: Estimated head radius (975.6 cm) is above the 99th percentile for adult head size.
  raw.plot_sensors(show_names=False)
/var/folders/0w/p5w49zc93976_p2hxj1zws2w0000gn/T/ipykernel_60715/1267379592.py:2: RuntimeWarning: (X, Y) fit (1833.6, 353.1) more than 20 mm from head frame origin
  raw.plot_sensors(show_names=False)


TypeError: montage must be an instance of DigMontage, got <class 'mne._fiff.meas_info.Info'> instead.

In [14]:
fig = mne.viz.plot_alignment(
    raw.info,              # raw object with the montage
    surfaces=['head'],     # Show the head surface (optional)
    coord_frame='head',    # Coordinates system (head)
    show_axes=True,        # Show coordinate axes
    dig=True,              # Plot the headshape points (extras)
    eeg=True,              # Plot the EEG electrodes
    meg=False              # Disable plotting MEG sensors (if not applicable)
)

ValueError: A head<->mri transformation matrix (trans) is required to plot head surfaces in head coordinates, `trans=None` is not allowed

In [13]:
raw.info.get_montage()

<DigMontage | 0 extras (headshape), 0 HPIs, 3 fiducials, 267 channels>